In [ ]:
######SOBOL SEQUENCE GENERATION

import numpy as np
from scipy.stats import qmc
import glob
import numpy as np
import json

with open('configAMPSIT.json') as config_file:
    config = json.load(config_file)

parameter_names = config['parameter_names']

# Parameters
DIM = len(parameter_names)
start = 100000
STEP = 1
endsim = config['totalsim']

# Generate Sobol sequence
sobol_seq = qmc.Sobol(d=DIM, scramble=True)

# Advance in the sequence until "start"
sobol_seq.fast_forward(start)

# Get the latest endsim samples
p1 = np.array([sobol_seq.random() for _ in range(endsim)])

# Re-scaling
MATRIX =config['MATRIX']

# Initialize MINN and MAXX with zeros
MINN = np.zeros(len(MATRIX))
MAXX = np.zeros(len(MATRIX))

# Calculate MINN and MAXX
for i in range(len(MATRIX)):
    if MATRIX[i][0] < 0:
        MINN[i] = MATRIX[i][0] + MATRIX[i][0] * MATRIX[i][1] / 100
        MAXX[i] = MATRIX[i][0] - MATRIX[i][0] * MATRIX[i][1] / 100
    else:
        MINN[i] = MATRIX[i][0] - MATRIX[i][0] * MATRIX[i][1] / 100
        MAXX[i] = MATRIX[i][0] + MATRIX[i][0] * MATRIX[i][1] / 100

X = np.zeros((endsim, DIM))

for i in range(DIM):
    X[:, i] = p1[:, 0, i] * (MAXX[i] - MINN[i]) + MINN[i]

np.savetxt(f'X_AMPSIT_{endsim}.txt', X, delimiter=' ', fmt='%0.5f')

